参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。
此时，我们的目标是找到使损失函数最小化的模型参数值。经过训练后，我们将需要使用这些参数来做出未来的预测。 
此外，有时我们希望提取参数，以便在其他环境中复用它们，将模型保存下来，以便它可以在其他软件中执行，或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作， 而忽略了操作参数的具体细节。 
本节，我们将介绍以下内容：
    访问参数，用于调试、诊断和可视化；
    参数初始化；
    在不同模型组件间共享参数。


具有单隐藏层的多层感知机

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0911],
        [0.2410]], grad_fn=<AddmmBackward0>)

参数访问

从已有模型中访问参数。 当通过Sequential类定义模型时， 我们可以通过索引来访问模型的任意层。 这就像模型是一个列表一样，每层的参数都在其属性中。 如下所示，我们可以检查第二个全连接层的参数。

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2142, -0.3081,  0.0971,  0.1836, -0.3224,  0.2437, -0.0050, -0.3100]])), ('bias', tensor([0.2819]))])


也可以类似于下面的方式来等价地查询

In [6]:
net.state_dict()['2.bias'].data

tensor([0.2819])

每个参数都是参数类的一个实例

例如访问第二层中的偏置参数，type返回类型，print(net[2].bias)直接返回这个参数类的实例，data返回数值

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2819], requires_grad=True)
tensor([0.2819])


参数是复合的对象，包含值、梯度和额外信息。 这就是我们需要显式参数值的原因。 除了值之外，我们还可以访问每个参数的梯度。 在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。

In [4]:
net[2].weight.grad == None

True

一次性访问所有参数

In [ ]:
# * 运算符将元组解包，便于打印
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


从嵌套块收集参数

查看如果将多个块相互嵌套，参数命名约定是如何工作的。 我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.2357],
        [0.2353]], grad_fn=<AddmmBackward0>)

通过输出来查看是如何进行工作的

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


也就是同样构成了这种分层嵌套的形式

最外层是rgnet = nn.Sequential(block2(), nn.Linear(4, 1))中的block2和Linear
而block2中就是嵌套生成的四个block1()

因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。 
也就是类似于多维数组的访问方式
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。

In [9]:
rgnet[0][1][0].bias.data

tensor([-0.1486,  0.4137, -0.1204, -0.2643,  0.3114,  0.0542, -0.1867, -0.3463])

参数初始化

深度学习框架提供默认随机初始化， 也允许我们创建自定义初始化方法， 满足我们通过其他规则实现初始化权重。

默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵， 这个范围是根据输入和输出维度计算出的。 PyTorch的nn.init模块提供了多种预置初始化方法。

调用内置初始化器

In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)  # 将所有权重参数初始化为标准差为0.01的正态分布
        nn.init.zeros_(m.bias)  # 将偏置参数设置为0
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0130,  0.0145,  0.0115, -0.0167]), tensor(0.))

In [11]:
# 也可以将所有参数初始化为给定的常数，比如初始化为1
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

还可以对某些块应用不同的初始化方法。 
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层， 然后将第三个神经网络层初始化为常量值42。

Xavier 初始化的核心思想是保持各层输入和输出的方差一致，这样信号既不会在前向传播时逐渐消失，也不会在反向传播时逐渐消失。

为此从从均值为 0、方差为:$\sigma^2 = \frac{2}{n_{in} + n_{out}}$的均匀分布或正态分布中抽取权重。

具体实现有两种常见变体：
    均匀分布版本：从均匀分布 $U[-a, a]$ 中采样，其中: $$a = \sqrt{\frac{6}{n_{in} + n_{out}}}$$
    正态分布版本：从正态分布 $N(0, \sigma^2)$ 中采样，其中: $$\sigma = \sqrt{\frac{2}{n_{in} + n_{out}}}$$


PyTorch中的实现
均匀分布版本
    nn.init.xavier_uniform_(tensor)
正态分布版本
    nn.init.xavier_normal_(tensor)


In [ ]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

# # 应用到整个网络
# net.apply(init_xavier)
# # 或者只应用到特定层
# nn.init.xavier_uniform_(net[0].weight)

tensor([ 0.5253, -0.4449, -0.2545, -0.5283])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [ ]:
# uniform_ 函数表示进行均匀分布初始化
# 下划线后缀表示这是一个原地操作(in-place operation)，会直接修改 m.weight 的值

def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)

        # 只保留绝对值大于等于5的元素,实现一种初始化
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-5.6208,  0.0000,  0.0000,  0.0000],
        [-0.0000, -0.0000,  7.7565, -0.0000]], grad_fn=<SliceBackward0>)

也可以直接设置参数值

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

参数绑定

用于希望在多个层间共享参数
可以定义一个稠密层,然后使用它的参数来设置另一个层的参数

共享参数通常可以节省内存，并在以下方面具有特定的好处：
    对于图像识别中的CNN，共享参数使网络能够在图像中的任何地方而不是仅在某个区域中查找给定的功能。
    对于RNN，它在序列的各个时间步之间共享参数，因此可以很好地推广到不同序列长度的示例。
    对于自动编码器，编码器和解码器共享参数。 在具有线性激活的单层自动编码器中，共享权重会在权重矩阵的不同隐藏层之间强制正交。

In [ ]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值，因此在修改其中一个参数后再次判断
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
